In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import gc
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 10
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


In [3]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error as mae

def Denseblock(n, activation=mish, bn=False):
    def f(x):
        x = Dense(n)(x)
        if bn:
            x = BatchNormalization()(x)
        x = Activation(activation)(x)
        return x
    return f

In [4]:
class GAN:
    def __init__(self, x_shape, y_shape): # latent = c_shape
        # data structure
        self.y_shape = y_shape
        self.x_shape = x_shape
        
        self.score = float('inf')
        self.mapes = []
        self.maes = []
        
        # build model
        self.gen = self.build_generator()
        self.disc = self.build_discriminator()
        
        inputs_x = Input(shape = self.x_shape[1:])
        inputs_y = Input(shape = self.y_shape[1:])
        
        pred_y = self.gen(inputs_x)

        self.disc.compile(optimizer=RMSprop(2e-4), loss='binary_crossentropy')
        self.disc.trainable = False

        valid = self.disc([inputs_x, pred_y])
        
        self.gan = Model([inputs_x, inputs_y], [valid, pred_y], name='GAN')
        self.gan.compile(optimizer=RMSprop(2e-4), loss=['binary_crossentropy']+['mae'], loss_weights=[1, 100])
        
    
    def build_generator(self):
        inputs = Input(shape = self.x_shape[1:])
        
        x = Denseblock(64)(inputs)
        x = Denseblock(32)(x)
        x = Denseblock(16)(x)

        outputs = Dense(self.y_shape[1])(x)
        
        gen = Model(inputs, outputs, name='generator')
        
        return gen
    
    def build_discriminator(self):
        inputs_y = Input(shape = self.y_shape[1:])
        inputs_x = Input(shape = self.x_shape[1:])
        
        inputs = Concatenate()([inputs_x, inputs_y])
        
        x = Denseblock(64)(inputs)
        x = Denseblock(32)(x)
        
        outputs = Dense(1, activation='sigmoid')(x)
        
        disc = Model([inputs_x, inputs_y], outputs, name='discriminator')
        
        return disc

    def train(self, x, y, epochs=10, batch_size=1024):
        X = x.copy()
        Y = y.copy()

        n_ = self.y_shape[0]//batch_size
        
        for e in range(epochs):
#             idx = np.random.randint(0, self.y_shape[0], batch_size)
            kf = KFold(n_, shuffle=True)
            tot_disc_loss = 0
            tot_gan_loss = 0
            tot_mse_loss = 0
            for _, idx in kf.split(x):
                real_label = np.zeros((len(idx), 1))
                fake_label = np.ones((len(idx), 1))
                train_x = x[idx]
                train_y = y[idx]

                fake = self.gen.predict(train_x)

                # train discriminator
                d_loss_real = self.disc.train_on_batch([train_x, train_y], real_label)
                d_loss_fake = self.disc.train_on_batch([train_x, fake], fake_label)
                d_loss = 0.5*np.add(d_loss_fake, d_loss_real)

                # train generator
                g_loss = self.gan.train_on_batch([train_x, train_y], [real_label] + [train_y])
                
                tot_disc_loss += d_loss
                tot_gan_loss += g_loss[1]
                tot_mse_loss += g_loss[2]
                
            print(f'iter:{e}')
            print(f'|disc_loss: {round(d_loss, 3)}, |gan_loss:, {round(g_loss[1], 3)} |mse_loss: {round(g_loss[2], 3)}')
            
            # val score example
            #mape_ = round(self.evaluate(val_X, val_y, mape), 3)
            mae_ = round(self.evaluate(val_X, val_y, mae), 3)
            print(f'validation || mae: {mae_}')
            print('#'*50)
            
            if self.score > mae_:
                self.score = mae_
                self.best_weights = self.gan.get_weights()
                
            #self.mapes.append(mape_)
            self.maes.append(mae_)
            
        self.gan.set_weights(self.best_weights)
            
            
    def evaluate(self, x, y, criterion):
        y_pred = self.gen.predict(x)
        res = criterion(y, y_pred)
        return res

In [5]:
from sklearn.model_selection import train_test_split

X = np.random.normal(0, 1, (5000, 120))
y = np.random.normal(0, 1, (5000, 3))

tr_X, val_X, tr_y, val_y = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [6]:
gan = GAN(tr_X.shape, tr_y.shape)

In [7]:
gan.train(tr_X, tr_y,
         epochs=20,
         batch_size=128)

iter:0
|disc_loss: 0.768, |gan_loss:, 0.797 |mse_loss: 0.851
validation || mae: 0.839
##################################################
iter:1
|disc_loss: 0.747, |gan_loss:, 0.725 |mse_loss: 0.82
validation || mae: 0.832
##################################################
iter:2
|disc_loss: 0.733, |gan_loss:, 0.738 |mse_loss: 0.788
validation || mae: 0.828
##################################################
iter:3
|disc_loss: 0.73, |gan_loss:, 0.713 |mse_loss: 0.824
validation || mae: 0.825
##################################################
iter:4
|disc_loss: 0.73, |gan_loss:, 0.716 |mse_loss: 0.784
validation || mae: 0.823
##################################################
iter:5
|disc_loss: 0.718, |gan_loss:, 0.74 |mse_loss: 0.792
validation || mae: 0.821
##################################################
iter:6
|disc_loss: 0.712, |gan_loss:, 0.712 |mse_loss: 0.795
validation || mae: 0.82
##################################################
iter:7
|disc_loss: 0.715, |gan_loss:, 0.696 |m

In [8]:
gan.gen.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                7744      
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0 

In [9]:
gan.disc.summary()

Model: "discriminator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 120)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 123)          0           input_3[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 64)           7936        concatenate[0][0]    